## lgb, binary, テーブルデータ

### データ読み込み

In [1]:
import pandas as pd
df = pd.read_csv("/mnt/sdb/SIGNATE_DATA/liver-disease-detection/train.csv").append(pd.read_csv("/mnt/sdb/SIGNATE_DATA/liver-disease-detection/test.csv"))

/tmp/ipykernel_45390/2919541427.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = pd.read_csv("/mnt/sdb/SIGNATE_DATA/liver-disease-detection/train.csv").append(pd.read_csv("/mnt/sdb/SIGNATE_DATA/liver-disease-detection/test.csv"))


In [2]:
df = df.reset_index(drop=False)
df.rename(columns={"disease": "target"}, inplace=True)

df["Gender"] = df["Gender"].map(lambda x: {"Male":1, "Female":0}[x])

In [3]:
df

,index,Age,Gender,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,target
0,0,59,1,0.786886,0.150498,220.178691,13.467617,21.729246,6.815731,3.112276,1.006802,0.0
1,1,69,1,1.003987,0.195625,221.218413,51.033462,64.752323,6.889608,3.051521,0.751346,1.0
2,2,65,1,0.657140,0.081287,320.770533,12.625011,30.613180,5.947767,2.489167,0.774952,0.0
3,3,65,1,0.906822,0.214173,369.278160,34.347597,54.510085,6.967183,3.613837,0.988155,1.0
4,4,22,0,1.734959,0.197706,222.782025,20.572891,170.010177,5.837537,3.068697,1.026654,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1195,345,65,0,0.913867,0.120555,214.354728,16.964820,28.723677,6.960649,3.505619,1.224655,NaN
1196,346,59,1,0.818573,0.206075,215.898729,14.733461,20.470517,7.531071,3.637508,1.282833,NaN
1197,347,25,1,0.787054,0.186129,214.167523,14.936585,20.589993,7.505128,3.619033,1.287256,NaN
1198,348,64,1,0.725194,0.191307,220.272862,17.532595,48.115014,7.559414,3.635871,1.263348,NaN


In [4]:
train = df.iloc[0:850, :]
test = df.iloc[850:, :]

In [5]:
from myutils import *

/home/marimo/poetry_projects/myfunctions/myutils.py:60: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  print("%s - done in %is"(title, (datetime.datetime.now() - t0).seconds))


*** id_name:index // label_name:target
*** Seed_no_torch *** 


In [6]:
lgb_config = {
    'lgb_params':{
                  'objective' : 'binary',
                  'metric' : 'binary_logloss',
                  'boosting': 'dart',
                  'max_depth' : -1,
                  'num_leaves' : 64,
                  'learning_rate' : 0.035,
                  'bagging_freq': 5,
                  'bagging_fraction' : 0.75,
                  'feature_fraction' : 0.05,
                  'min_data_in_leaf': 256,
                  'max_bin': 63,
                  'min_data_in_bin': 256,
                  'tree_learner': 'serial',
                  'boost_from_average': 'false',
                  'lambda_l1' : 0.1,
                  'lambda_l2' : 30,
                  'num_threads': 24,
                  'verbosity' : 1,
    },
    'feature_name':[col for col in df.columns if col not in ["target"]],
    'rounds':1,
    'early_stopping_rounds':None,
    'verbose_eval':50,
    'folds':5,
    'seed':args.seed
}

In [7]:
Lgb_train_and_predict(train, test, lgb_config)

[LightGBM] [Info] Number of positive: 301, number of negative: 379
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000413 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31
[LightGBM] [Info] Number of data points in the train set: 680, number of used features: 10
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
*** - fold0 valid metric: 0.693147

[LightGBM] [Info] Number of positive: 302, number of negative: 378
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000344 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31
[LightGBM] [Info] Number of data points in the train set: 680, number of used features: 10
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


cp: cannot stat './*.sh': No such file or directory
/home/marimo/poetry_projects/myfunctions/myutils.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oof[label_name] = 0
/home/marimo/poetry_projects/myfunctions/.venv/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/home/marimo/poetry_projects/myfunctions/.venv/lib/python3.8/site-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM.

(     index  target
 0        0     0.5
 1        1     0.5
 2        2     0.5
 3        3     0.5
 4        4     0.5
 ..     ...     ...
 845    845     0.5
 846    846     0.5
 847    847     0.5
 848    848     0.5
 849    849     0.5
 
 [850 rows x 2 columns],
       index  prediction
 850       0         0.5
 851       1         0.5
 852       2         0.5
 853       3         0.5
 854       4         0.5
 ...     ...         ...
 1195    345         0.5
 1196    346         0.5
 1197    347         0.5
 1198    348         0.5
 1199    349         0.5
 
 [350 rows x 2 columns],
 (0.6931471805599453, 0.6931471805599453))